# Case do Paraná Banco (PB)

## Contexto: Crédito Consignado

### Objetivo: Recomendação para maximizar a receita

* **Descrição do problema**: Quando o cliente morre, surge uma dívida que implica em prejuízo financeiro para o banco. Baseado nisso, como desenvolver uma estratégia para maximizar o ganho?

In [1]:
#Bibliotecas
import pandas as pd
import numpy as np
from scipy import stats
##Eliminar os warnings
import warnings
warnings.filterwarnings("ignore")
##Ver todas as colunas do data frame
pd.set_option('display.max_columns', None)
##Ver todas as linhas do data frame
pd.set_option('display.max_rows', None)

In [2]:
#Fórmula do KS
##Utilizar o KS para calcular a relação entre óbito e as variáveis quantitativas
def calcular_ks_2samp(df, alvo, escore):
    bons_escore = df.loc[df[alvo] == 0, escore].rename('bons')
    maus_escore = df.loc[df[alvo] == 1, escore].rename('maus')
    return stats.ks_2samp(bons_escore, maus_escore).statistic

# 1 - Visão geral dos dados

In [3]:
import chardet

#Detectar a codificação do arquivo
with open('case_ds.csv', 'rb') as f:
    result = chardet.detect(f.read())

#Ler o arquivo
df = pd.read_csv('case_ds.csv', sep=';', encoding=result['encoding'])
#Visualização
df.head(3)

,codigocliente,AnoMesBase,Aposentadoria_idade,Aposentadoria_invalidez,Aposentadoria_tempo_contribuicao,Pensao_morte,Auxilio,Beneficios_assistenciais,Diversos_e_uniao,Obito,IdadeAnos,DataNascimento,Regiao,ClienteEstado,Sexo,EstadoCivil,distinct_operacoes,VlrPrincipal_Inicial,VlrPrincipal_Atual,VlrPreju¡zo,sexo,duration,VlrReceitaJuros,VlrReceitaJuros_74
0,1,202412,1,0,0,0,0,0,0,0,74,1950-09-07T00:00:00.000Z,Sudeste,SP,M,CASADO,2,39675.63,39174.04,0.0,M,73.0,67561.44,4310.82
1,2,202412,0,0,1,0,0,0,0,0,76,1948-07-13T00:00:00.000Z,Sudeste,SP,M,CASADO,2,2631.99,1889.23,0.0,M,36.0,17265.99,13262.13
2,3,202412,0,0,1,0,0,0,0,0,77,1947-02-04T00:00:00.000Z,Sudeste,SP,M,CASADO,1,5310.62,4685.68,0.0,M,11.0,6289.04,1390.65


In [4]:
##################################
#Características da base
##################################
df.info() 
#Total de linhas: 16.849
#As variáveis Sexo e duration (quase 100% de preenchimento) não tem 100% de preenchimento
#Existem duas variáveis que aparentam ser iguais: Sexo e sexo  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16849 entries, 0 to 16848
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   codigocliente                     16849 non-null  int64  
 1   AnoMesBase                        16849 non-null  int64  
 2   Aposentadoria_idade               16849 non-null  int64  
 3   Aposentadoria_invalidez           16849 non-null  int64  
 4   Aposentadoria_tempo_contribuicao  16849 non-null  int64  
 5   Pensao_morte                      16849 non-null  int64  
 6   Auxilio                           16849 non-null  int64  
 7   Beneficios_assistenciais          16849 non-null  int64  
 8   Diversos_e_uniao                  16849 non-null  int64  
 9   Obito                             16849 non-null  int64  
 10  IdadeAnos                         16849 non-null  int64  
 11  DataNascimento                    16849 non-null  object 
 12  Regi

In [5]:
#Verificar se existe algum código de cliente repetido
df['codigocliente'].unique().shape[0] #Cada linha representa um único cliente 16.849 (c)

16849

## Comparativo entre as variáveis Sexo e sexo

In [6]:
validacao_sexo = df[['codigocliente', 'Sexo', 'sexo']]
#Visualização
validacao_sexo.head(3)

,codigocliente,Sexo,sexo
0,1,M,M
1,2,M,M
2,3,M,M


In [7]:
#Adicionar a variável flag_validacao
validacao_sexo['flag_validacao'] = np.where(validacao_sexo['Sexo']  ==  validacao_sexo['sexo'], 1,0)
#Visualização
validacao_sexo.head(3)

,codigocliente,Sexo,sexo,flag_validacao
0,1,M,M,1
1,2,M,M,1
2,3,M,M,1


In [8]:
##Observações iguais
validacao_sexo['flag_validacao'].sum() #14573. As diferenças ocorrem justamente onde não tem preenchimento na variável Sexo.

np.int64(14573)

* **Decisão 1**: Considerar a variável **sexo** no lugar da variável **Sexo**.

In [9]:
#Remover a variável Sexo do data frame base
df = df.drop('Sexo', axis = 1)

# 2 - Data Wrangling

In [10]:
#Casting: Colocar DataNascimento como data
df['DataNascimento'] = pd.to_datetime(df.DataNascimento).dt.tz_localize(None)

In [11]:
#Casting: Colocar AnoMesBase como string
df = df.astype({'AnoMesBase': 'string'})

In [12]:
#Renomear a variável VlrPreju¡zo para valor_prejuizo
df.rename(columns = {'VlrPreju¡zo':'valor_prejuizo'}, inplace = True)

In [13]:
#Adicionar a variável lucro
df['lucro'] = df['VlrReceitaJuros'] * 0.0016 #lucro representa 0,16% da receita de juros

# 3 - Caracterização dos clientes quanto a remuneração recebida

In [14]:
#Idade
df.loc[df['Aposentadoria_idade'] == 1].shape[0] #7361

7361

In [15]:
#Invalidez
df.loc[df['Aposentadoria_invalidez'] == 1].shape[0] #1351

1351

In [16]:
#Tempo de contribuição
df.loc[df['Aposentadoria_tempo_contribuicao'] == 1].shape[0] #4426

4426

In [17]:
#Pensão por morte
df.loc[df['Pensao_morte'] == 1].shape[0] #5583

5583

In [18]:
#Auxilio
df.loc[df['Auxilio'] == 1].shape[0] #0

0

In [19]:
#Benefício assistencial
df.loc[df['Beneficios_assistenciais'] == 1].shape[0] #0

0

In [20]:
#Diversos e união
df.loc[df['Diversos_e_uniao'] == 1].shape[0] #0

0

**OBS**: 
1. Nenhum cliente recebe algum tipo de benefício/auxílio.
2. Ao somar individualmente cada grupo, o total ultrapassa o volume de clientes. Portanto, existe intersecção, ou seja, um cliente pode está associado a mais de um grupo.

In [21]:
#O fato da diferença ser diferente de 0 implica que existe intersecção.
(df.loc[df['Aposentadoria_idade'] == 1].shape[0] + df.loc[df['Aposentadoria_invalidez'] == 1].shape[0]+
df.loc[df['Aposentadoria_tempo_contribuicao'] == 1].shape[0] + df.loc[df['Pensao_morte'] == 1].shape[0]) - df.shape[0] #Diferença é de 1872 

1872

### Agregação

In [22]:
# Lista das colunas binárias referente a caracterização do público
colunas_binarias = ['Aposentadoria_idade', 'Aposentadoria_invalidez', 'Aposentadoria_tempo_contribuicao', 'Pensao_morte', 'Auxilio',
                   'Beneficios_assistenciais', 'Diversos_e_uniao']

# Cria uma nova coluna 'publico_alvo' que combina os nomes das colunas onde o valor é 1
df['publico_alvo'] = df[colunas_binarias].apply(lambda row: '-'.join(row.index[row == 1]) if any(row == 1) else 'outro_tipo_cliente', axis=1)

# 4 - Volumetria por safra

In [23]:
#Volumetria por safra
vol_safra = df.groupby('AnoMesBase').size().reset_index(name='volumetria')
#Renomeia as colunas
vol_safra.columns = ['safra', 'volumetria']
vol_safra.head(12) #Os dados estão concentrados em dezembro de 2024 (95% dos casos)

,safra,volumetria
0,202401,59
1,202402,48
2,202403,67
3,202404,72
4,202405,70
5,202406,122
6,202407,94
7,202408,79
8,202409,93
9,202410,72


## Casos de óbito

In [60]:
#Casos de óbito
df.loc[df.Obito == 1].shape[0] #984

984

In [59]:
#Taxa de óbito
round(100 * df['Obito'].value_counts(True),2) #5.84%

Obito
0    94.16
1     5.84
Name: proportion, dtype: float64

In [61]:
#Número de óbitos por mês
volumetria_nobito_obito_safra = df.groupby('AnoMesBase')['Obito'].value_counts().unstack()
#Visualização
volumetria_nobito_obito_safra

Obito,0,1
AnoMesBase,,
202401,NaN,59.0
202402,NaN,48.0
202403,NaN,67.0
202404,NaN,72.0
202405,NaN,70.0
202406,NaN,122.0
202407,NaN,94.0
202408,NaN,79.0
202409,NaN,93.0


# 5 - Caracterização dos clientes

## Distribuição por estado

In [24]:
#Distribuição por estado
df_estado = round(100 * df['ClienteEstado'].value_counts(True),2)
#Transformar num data frame
##Converte a série em um DataFrame e renomeia as colunas
df_estado = df_estado.reset_index()
df_estado.columns = ['estado', 'percentual']
#Visualização
df_estado.head(3)

,estado,percentual
0,PR,24.14
1,SP,22.29
2,RS,15.62


In [25]:
df_estado.loc[df_estado.percentual > 5] #Pareto: 86% do fluxo de operações é concentrado em 5 estados

,estado,percentual
0,PR,24.14
1,SP,22.29
2,RS,15.62
3,RJ,13.21
4,SC,10.63


In [26]:
round(24.14 + 22.29 + 15.62 + 13.21 + 10.63,0) #86% 

86.0

In [27]:
#Salvar por estado
df_estado.to_csv('dados_estado.csv', header=True, index=False, sep=';')

## Distribuição por região

In [28]:
#Distribuição por região
df_regiao = round(100 * df['Regiao'].value_counts(True),2)
#Transformar num data frame
##Converte a série em um DataFrame e renomeia as colunas
df_regiao = df_regiao.reset_index()
df_regiao.columns = ['regiao', 'percentual']
#Visualização
df_regiao.head(5) #Pareto: Sul + Suldeste = 91% dos clientes

,regiao,percentual
0,Sul,50.39
1,Sudeste,40.95
2,Nordeste,6.48
3,Centro-Oeste,1.61
4,Norte,0.58


## Distribuição por estado civil

In [29]:
#Distribuição por estado civil
df_ec = round(100 * df['EstadoCivil'].value_counts(True),2)
#Transformar num data frame
##Converte a série em um DataFrame e renomeia as colunas
df_ec = df_ec.reset_index()
df_ec.columns = ['estado_civil', 'percentual']
#Visualização
df_ec.head(5) 

,estado_civil,percentual
0,OUTROS,48.73
1,SOLTEIRO,26.73
2,CASADO,13.50
3,VIUVO,9.97
4,DIVORCIADO,0.72


## Distribuição por idade

In [30]:
df['IdadeAnos'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) #Público Idoso: Acima de 70 anos. 
#Média: 76
#Mediana: 76
#Mínimo:74
#Máximo: 83
#Pouco variabilidade e concentrado em idosos

count    16849.000000
mean        76.279661
std          1.837541
min         74.000000
10%         74.000000
20%         74.000000
30%         75.000000
40%         75.000000
50%         76.000000
60%         77.000000
70%         77.000000
80%         78.000000
90%         79.000000
max         83.000000
Name: IdadeAnos, dtype: float64

### Criação da variável idade_faixas.

* Colocar idade segmentada em faixas: **74**, **75**, **76**, **77**, **78 - 79**, **80 - MAX**.

In [31]:
#idade_faixas
condicoes_idade = [
    df['IdadeAnos'] <= 74,
    df['IdadeAnos'] == 75,
    df['IdadeAnos'] == 76,
    df['IdadeAnos'] == 77,
    df['IdadeAnos'].isin([78, 79]),
    df['IdadeAnos'] > 79
]

rotulos_idade = ['F74', 'F75', 'F76', 'F77', 'F78-79', 'F80-MAX']

# Cria a variável categórica idade_faixas
df['idade_faixas'] = np.select(condicoes_idade, rotulos_idade, default='Outro')

## Distribuição por operação

In [32]:
#Distribuição por operação
df_operacao = round(100 * df['distinct_operacoes'].value_counts(True),2)
#Transformar num data frame
##Converte a série em um DataFrame e renomeia as colunas
df_operacao = df_operacao.reset_index()
df_operacao.columns = ['operacao', 'percentual']
#Visualização
df_operacao.head(3)

,operacao,percentual
0,1,28.99
1,2,17.09
2,3,11.55


In [33]:
#Pareto da operação
df_operacao.loc[df_operacao.percentual > 5] #Pareto: 80% do fluxo de operações é concentrado em até 6 operações

,operacao,percentual
0,1,28.99
1,2,17.09
2,3,11.55
3,4,9.09
4,5,7.18
5,6,6.12


In [34]:
round(28.99 + 17.09 + 11.55 + 9.09 + 7.18 + 6.12,0) #80%

80.0

### Criação da variável operacao_faixas.

* Colocar operação segmentada em faixas: **1**, **2**, **3**, **4**, **5**, **6**, **7 - MAX**.

In [35]:
#operacao_faixas
condicoes_operacao = [
    df['distinct_operacoes'] <= 1,
    df['distinct_operacoes'] == 2,
    df['distinct_operacoes'] == 3,
    df['distinct_operacoes'] == 4,
    df['distinct_operacoes'] == 5,
    df['distinct_operacoes'] == 6,
    df['IdadeAnos'] > 6
]

rotulos_operacao = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7-MAX']

# Cria a variável categórica idade_faixas
df['operacao_faixas'] = np.select(condicoes_operacao, rotulos_operacao, default='Outro')

## Distribuição por sexo

In [36]:
#Distribuição por sexo
df_sexo = round(100 * df['sexo'].value_counts(True),2)
#Transformar num data frame
##Converte a série em um DataFrame e renomeia as colunas
df_sexo = df_sexo.reset_index()
df_sexo.columns = ['Sexo', 'Percentual']
#Visualização
df_sexo.head(3) #54 (F) vs 46 (M)

,Sexo,Percentual
0,F,54.15
1,M,45.85


## Tipo de remuneração dos clientes

In [37]:
#Distribuição por público-alvo
df_publico = round(100 * df['publico_alvo'].value_counts(True),2)
#Transformar num data frame
##Converte a série em um DataFrame e renomeia as colunas
df_publico = df_publico.reset_index()
df_publico.columns = ['publico_alvo', 'percentual']
#Visualização
df_publico.head(3)

,publico_alvo,percentual
0,Aposentadoria_idade,35.06
1,Aposentadoria_tempo_contribuicao,23.82
2,Pensao_morte,20.85


In [38]:
#Salvar por publico_alvo
df_publico.to_csv('dados_publico.csv', header=True, index=False, sep=';')

## Duration

* Tempo de relacionamento do cliente com o banco.

In [39]:
#Suposição: A variável está definida em mês
df['duration'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count    16843.000000
mean        66.711690
std         47.684823
min          1.000000
10%         11.000000
20%         25.000000
30%         43.000000
40%         52.000000
50%         57.000000
60%         62.000000
70%         71.000000
80%         95.000000
90%        146.000000
max        191.000000
Name: duration, dtype: float64

## Lucro

In [40]:
#Empréstimos que deram prejuízo
df_lucro = df.loc[df.Obito == 0]
#Análise
df_lucro['lucro'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) #Lucro médio: 24 reais

count    15865.000000
mean        24.404452
std         35.061996
min        -17.826048
10%          1.023325
20%          2.707245
30%          5.372602
40%          9.358490
50%         14.321296
60%         19.559850
70%         26.287184
80%         36.936368
90%         57.458083
max        760.776512
Name: lucro, dtype: float64

In [42]:
#Análise 1
df_lucro.loc[df.lucro<0].shape[0] #6

6

In [43]:
#Análise 2
df_lucro.loc[df.lucro<0].head(6)

,codigocliente,AnoMesBase,Aposentadoria_idade,Aposentadoria_invalidez,Aposentadoria_tempo_contribuicao,Pensao_morte,Auxilio,Beneficios_assistenciais,Diversos_e_uniao,Obito,IdadeAnos,DataNascimento,Regiao,ClienteEstado,EstadoCivil,distinct_operacoes,VlrPrincipal_Inicial,VlrPrincipal_Atual,valor_prejuizo,sexo,duration,VlrReceitaJuros,VlrReceitaJuros_74,lucro,publico_alvo,idade_faixas,operacao_faixas
8459,8460,202412,1,0,0,1,0,0,0,0,77,1947-05-27 00:00:00,Sudeste,SP,OUTROS,4,10149.59,5146.29,0.0,F,54.0,-11141.28,4596.99,-17.826048,Aposentadoria_idade-Pensao_morte,F77,F4
8766,8767,202412,0,0,1,0,0,0,0,0,77,1947-05-18 00:00:00,Sudeste,RJ,OUTROS,5,16183.80,10439.52,0.0,F,60.0,-4271.34,9218.59,-6.834144,Aposentadoria_tempo_contribuicao,F77,F5
8909,8910,202412,1,0,0,0,0,0,0,0,77,1947-08-18 00:00:00,Sul,SC,SOLTEIRO,1,615.39,198.43,0.0,F,57.0,-125.27,-342.46,-0.200432,Aposentadoria_idade,F77,F1
11195,11196,202412,0,0,0,1,0,0,0,0,76,1948-07-10 03:00:00,Nordeste,BA,OUTROS,3,2751.34,2485.90,0.0,F,57.0,-276.35,959.17,-0.442160,Pensao_morte,F76,F3
11833,11834,202412,0,0,1,1,0,0,0,0,74,1950-07-08 00:00:00,Sul,PR,OUTROS,4,1789.05,794.95,0.0,F,58.0,-8596.86,92.46,-13.754976,Aposentadoria_tempo_contribuicao-Pensao_morte,F74,F4
15499,15500,202412,1,0,0,0,0,0,0,0,78,1946-04-28 00:00:00,Nordeste,PB,OUTROS,1,989.36,268.31,0.0,F,61.0,-3293.73,-3366.20,-5.269968,Aposentadoria_idade,F78-79,F1


## Variáveis financeiras

### 1 - Prejuízo

* **Nota**: O prejuízo só ocorre quando óbito é igual a 1.

In [47]:
df['valor_prejuizo'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) #574 reais

count    16849.000000
mean       574.087813
std       3711.851332
min          0.000000
10%          0.000000
20%          0.000000
30%          0.000000
40%          0.000000
50%          0.000000
60%          0.000000
70%          0.000000
80%          0.000000
90%          0.000000
max      76651.660000
Name: valor_prejuizo, dtype: float64

In [48]:
#Validado - não tem prejuízo quando Obito é 0 
df.query('Obito == 0')['valor_prejuizo'].describe() 

count    15865.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: valor_prejuizo, dtype: float64

In [49]:
#Total de óbitos = 984
df.query('Obito == 1')['valor_prejuizo'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) 

count      984.000000
mean      9830.086951
std      12044.237451
min         11.090000
10%        530.110000
20%       1124.812000
30%       2030.294000
40%       3245.562000
50%       5376.475000
60%       8142.364000
70%      11256.539000
80%      15931.612000
90%      25562.321000
max      76651.660000
Name: valor_prejuizo, dtype: float64

In [51]:
#Prejuízo acumulado
round(df.query('Obito == 1')['valor_prejuizo'].sum(),0) #9.672.806

np.float64(9672806.0)

In [52]:
#Valor do principal acumulado
round(df.query('Obito == 1')['VlrPrincipal_Inicial'].sum(),0) #13.709.805

np.float64(13709805.0)

In [54]:
#Percentual do prejuízo sobre o total
round(df.query('Obito == 1')['valor_prejuizo'].sum()/df.query('Obito == 1')['VlrPrincipal_Inicial'].sum(),4) * 100 #70.55

np.float64(70.55)

### 2 - Distribuição por Valor Principal no início do contrato

* Valor do empréstimo.

In [55]:
df['VlrPrincipal_Inicial'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) #Empréstimo médio: 13698

count     16849.000000
mean      13697.645292
std       15335.393776
min          33.140000
10%        1163.472000
20%        2162.094000
30%        3641.226000
40%        5871.028000
50%        8965.450000
60%       12275.022000
70%       16304.016000
80%       21097.780000
90%       33116.078000
max      163665.860000
Name: VlrPrincipal_Inicial, dtype: float64

In [57]:
round(df['VlrPrincipal_Inicial'].sum(),0)#230.791.626

np.float64(230791626.0)

In [80]:
#Empréstimo dos que foram óbito
df.query('Obito == 1')['VlrPrincipal_Inicial'].describe() 

count      984.000000
mean     13932.728984
std      14988.507484
min         40.510000
25%       3061.557500
50%       9501.205000
75%      18317.915000
max      93582.930000
Name: VlrPrincipal_Inicial, dtype: float64

### Criação da variável emprestimo_faixas.

In [63]:
#emprestimo_faixas
condicoes_emprestimo = [
    df['VlrPrincipal_Inicial'] <= 10000,
   (df['VlrPrincipal_Inicial'] > 10000) & (df['VlrPrincipal_Inicial'] <= 20000),
    df['VlrPrincipal_Inicial'] > 20000
]

rotulos_emprestimo= ['ATE10000', '10000-20000', 'M20000']

# Cria a variável categórica idade_faixas
df['emprestimo_faixas'] = np.select(condicoes_emprestimo, rotulos_emprestimo, default='Outro')

### 3 - Distribuição por Valor Principal no período atual

In [58]:
df['VlrPrincipal_Atual'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count     16849.000000
mean       9536.307804
std       12544.434501
min           0.000000
10%         382.498000
20%        1011.544000
30%        1742.670000
40%        2892.010000
50%        4690.120000
60%        7263.764000
70%       10915.578000
80%       15897.612000
90%       24057.004000
max      142758.660000
Name: VlrPrincipal_Atual, dtype: float64

In [66]:
#Operações com prejuízo
##Visualização 
df.query('Obito == 1')[['codigocliente', 'duration', 'Obito', 'VlrPrincipal_Inicial', 'VlrPrincipal_Atual', 'valor_prejuizo', 'lucro']].head(4)

,codigocliente,duration,Obito,VlrPrincipal_Inicial,VlrPrincipal_Atual,valor_prejuizo,lucro
17,18,55.0,1,32121.92,19486.66,19486.66,42.635648
27,28,80.0,1,36308.18,21992.68,21992.68,55.971200
53,54,176.0,1,19488.54,17462.08,17462.08,56.742240
56,57,12.0,1,5494.72,4671.95,4671.95,2.156848


In [78]:
df[(df['Obito'] == 1) & (df['AnoMesBase'] == '202402') & (df['VlrPrincipal_Atual'] != df['valor_prejuizo'])][['AnoMesBase','codigocliente', 'duration', 'Obito', 'VlrPrincipal_Inicial', 'VlrPrincipal_Atual', 'valor_prejuizo', 'lucro']].head()

,AnoMesBase,codigocliente,duration,Obito,VlrPrincipal_Inicial,VlrPrincipal_Atual,valor_prejuizo,lucro
8344,202402,8345,145.0,1,7215.62,0.0,5736.58,9.726944
8591,202402,8592,41.0,1,16119.95,0.0,15529.07,11.152976
8914,202402,8915,48.0,1,5668.46,0.0,1947.61,5.171936
9080,202402,9081,160.0,1,9656.25,0.0,7231.17,72.900720
9703,202402,9704,46.0,1,5648.70,0.0,4802.36,6.141456


# 6 - Análise bivariada:  Relação entre óbito e as demais variáveis


## Variáveis qualitativas

In [79]:
colunas_qualitativas = df.select_dtypes(include= ['object', 'category']).columns
#Visualização
colunas_qualitativas

Index(['Regiao', 'ClienteEstado', 'EstadoCivil', 'sexo', 'publico_alvo',
       'idade_faixas', 'operacao_faixas', 'emprestimo_faixas'],
      dtype='object')

### Comparar a taxa de óbito para todas as classes das variáveis categóricas

In [81]:
#Iterando sobre as colunas categóricas
for coluna in colunas_qualitativas:
    print(f"Análise bivariada {coluna}:")
    summary = df.groupby([coluna])['Obito'].agg(['mean'])
    summary.columns = ['Média']
    print(summary)
    print('\n')

Análise bivariada Regiao:
                 Média
Regiao                
Centro-Oeste  0.080882
Nordeste      0.080660
Norte         0.041237
Sudeste       0.061603
Sul           0.052415


Análise bivariada ClienteEstado:
                  Média
ClienteEstado          
AL             0.153846
AM             0.000000
BA             0.089239
CE             0.057592
DF             0.067568
ES             0.080808
GO             0.089888
MA             0.000000
MG             0.067155
MS             0.083333
MT             0.000000
PA             0.045455
PB             0.100000
PE             0.081395
PI             0.045455
PR             0.046706
RJ             0.067835
RN             0.100000
RO             0.000000
RR             0.000000
RS             0.058533
SC             0.056393
SE             0.079545
SP             0.056192
TO             0.041667


Análise bivariada EstadoCivil:
                Média
EstadoCivil          
CASADO       0.045734
DESQUITADO   0.065574
DIVORCIAD

## Variáveis quantitativas

In [82]:
#Lista de colunas quantitativas
colunas_quantitativas = df.select_dtypes(include= ['int', 'float']).columns

In [83]:
#Lista de colunas que você quer remover
colunas_para_remover = ['codigocliente', 'Aposentadoria_idade', 'Aposentadoria_invalidez', 
                        'Aposentadoria_tempo_contribuicao', 'Pensao_morte', 'Auxilio', 
                        'Beneficios_assistenciais', 'Diversos_e_uniao', 'Obito']

### Comparar a variável óbito afeta a distribuição entre as variáveis quantiativas

In [84]:
#Colunas relevantes
colunas_quantitativas = colunas_quantitativas.difference(colunas_para_remover)
print(colunas_quantitativas)

Index(['IdadeAnos', 'VlrPrincipal_Atual', 'VlrPrincipal_Inicial',
       'VlrReceitaJuros', 'VlrReceitaJuros_74', 'distinct_operacoes',
       'duration', 'lucro', 'valor_prejuizo'],
      dtype='object')


In [87]:
#Iterando sobre as colunas categóricas
for coluna in colunas_quantitativas:
    print(f"Análise bivariada {coluna}:")
    summary = df.groupby(['Obito'])[coluna].agg(['mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
    summary.columns = ['Média', 'Mediana', 'Desvio Padrão', 'Percentil 25', 'Percentil 75']
    print(summary)
    print('\n')

Análise bivariada IdadeAnos:
           Média  Mediana  Desvio Padrão  Percentil 25  Percentil 75
Obito                                                               
0      76.261330     76.0       1.831060          75.0          78.0
1      76.575203     76.0       1.915943          75.0          78.0


Análise bivariada VlrPrincipal_Atual:
             Média  Mediana  Desvio Padrão  Percentil 25  Percentil 75
Obito                                                                 
0      9770.065606  4928.04   12615.715622       1471.88    13447.1600
1      5767.438364   681.59   10652.650835          0.00     6779.2725


Análise bivariada VlrPrincipal_Inicial:
              Média   Mediana  Desvio Padrão  Percentil 25  Percentil 75
Obito                                                                   
0      13683.064621  8899.930   15356.994634     2800.5100     18651.420
1      13932.728984  9501.205   14988.507484     3061.5575     18317.915


Análise bivariada VlrReceitaJuros:


In [85]:
tabela_numerica = df[['Obito', 'IdadeAnos', 'VlrPrincipal_Atual', 'VlrPrincipal_Inicial',
       'VlrReceitaJuros', 'VlrReceitaJuros_74', 'distinct_operacoes', 'lucro',
       'duration', 'valor_prejuizo']]

In [88]:
#Criação de um dicionário
dicionario = {}
#Aplicação de um laço para o cálculo do KS2 da lista de variáveis em colunas_var_quant
for coluna in colunas_quantitativas:
   resultado = calcular_ks_2samp(tabela_numerica,'Obito', coluna)
   dicionario[coluna] = resultado

In [89]:
#Dicionário - inclui a chave (variável) e o valor (KS2)
dicionario #Valor do principal

{'IdadeAnos': np.float64(0.06609867556286653),
 'VlrPrincipal_Atual': np.float64(0.4378267854534833),
 'VlrPrincipal_Inicial': np.float64(0.030475441927441616),
 'VlrReceitaJuros': np.float64(0.03565705559356258),
 'VlrReceitaJuros_74': np.float64(0.02913844967318252),
 'distinct_operacoes': np.float64(0.03461786311843573),
 'duration': np.float64(nan),
 'lucro': np.float64(0.03565705559356258),
 'valor_prejuizo': np.float64(1.0)}

In [90]:
df_duration = df.dropna(subset = ['duration'])
calcular_ks_2samp(df_duration, 'Obito', 'duration') #11%

np.float64(0.11532169757997723)

# 7 - Grupo de risco

In [101]:
g_risco = df.loc[(df.Regiao == 'Centro-Oeste') | (df.Regiao == 'Nordeste') | 
(df.ClienteEstado == 'ES') | (df.EstadoCivil == 'DESQUITADO') | (df.idade_faixas == 'F80-MAX')| 
(df.publico_alvo == 'Aposentadoria_invalidez') | (df.publico_alvo == ' Aposentadoria_invalidez-Pensao_morte') | 
(df.publico_alvo == 'outro_tipo_cliente')]
#Volumetria
g_risco.shape[0]

3545

In [107]:
#Visualização
g_risco.head(3)

,codigocliente,AnoMesBase,Aposentadoria_idade,Aposentadoria_invalidez,Aposentadoria_tempo_contribuicao,Pensao_morte,Auxilio,Beneficios_assistenciais,Diversos_e_uniao,Obito,IdadeAnos,DataNascimento,Regiao,ClienteEstado,EstadoCivil,distinct_operacoes,VlrPrincipal_Inicial,VlrPrincipal_Atual,valor_prejuizo,sexo,duration,VlrReceitaJuros,VlrReceitaJuros_74,lucro,publico_alvo,idade_faixas,operacao_faixas,emprestimo_faixas
4,5,202412,0,0,1,0,0,0,0,0,80,1944-10-07,Nordeste,BA,OUTROS,3,13777.43,2885.31,0.0,M,149.0,27947.43,10909.10,44.715888,Aposentadoria_tempo_contribuicao,F80-MAX,F3,10000-20000
8,9,202412,1,0,0,0,0,0,0,0,80,1944-10-11,Sul,PR,CASADO,8,13942.51,3208.81,0.0,M,62.0,10710.90,10710.90,17.137440,Aposentadoria_idade,F80-MAX,F7-MAX,10000-20000
11,12,202412,1,0,0,0,0,0,0,0,80,1944-03-08,Nordeste,BA,SOLTEIRO,1,798.80,45.24,0.0,M,71.0,8252.69,8252.69,13.204304,Aposentadoria_idade,F80-MAX,F1,ATE10000


In [106]:
#Representatividade
round(100 * g_risco.shape[0]/df.shape[0],0)

21.0

In [103]:
#Taxa de óbito
round(100 * g_risco['Obito'].value_counts(True),2) #8.35

Obito
0    91.65
1     8.35
Name: proportion, dtype: float64

In [108]:
g_risco['VlrPrincipal_Inicial'].mean()

np.float64(10280.853963328631)

In [111]:
round(g_risco['valor_prejuizo'].sum(),0)

np.float64(2051837.0)

In [112]:
round(df['valor_prejuizo'].sum(),0)

np.float64(9672806.0)

In [113]:
round(g_risco['valor_prejuizo'].sum(),0)/round(df['valor_prejuizo'].sum(),0)

np.float64(0.21212427913885587)

In [110]:
g_risco['valor_prejuizo'].mean()

np.float64(578.7974160789845)